In [1]:
# GPTv2 모델 사용
# TFGPT2LMHeadModel.from_pretrained('GPT 모델 이름') : 두개의 문장이 이어지는 문장인지를 판단하는 GPT 모델 로드
# AutoTokenizer.from_pretrained('GPT 모델 이름') : 위 로드된 모델이 학습되었을 당시에 사용된 토크나이저를 로드

import numpy as np
import random
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

# GPT가 생성할 문장의 방향성을 알려주기 위한 시작 문자열
sent = '예수님'

# 텍스트 시퀀스를 정수 시퀀스로 변환
input_ids = tokenizer.encode(sent)
input_ids = tf.convert_to_tensor([input_ids])
print(input_ids)

# 정수 시퀀스를 입력받아 GPT가 이어서 문장을 생성 : 약 20초 걸림 (using 1 cpu)
output = model.generate(input_ids, # a tensor containing the input sequence encoded as integer IDs
                        max_length=128, # the maximum length of the generated sequence, in terms of tokens
                        repetition_penalty=2.0, # avoiding repeated tokens (higher value means more diverse output)
                        use_cache=True) # enables or disables the use of the model's internal cache (repetitive output)

output_ids = output.numpy().tolist()[0]
print(output_ids)

# 정수 시퀀스를 텍스트 시퀀스로 변환
tokenizer.decode(output_ids)

2023-03-07 08:07:46.332150: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-07 08:07:46.332178: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-07 08:07:49.915734: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-07 08:07:49.915757: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-07 08:07:49.915772: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-255-120-161): /proc/driver/nvidia/version does not exist
2023-03-07 08:07:49.915992: I tensorflow/core/platform/cpu_

tf.Tensor([[17835  7177]], shape=(1, 2), dtype=int32)
[17835, 7177, 6995, 16098, 7281, 9885, 13358, 10010, 6947, 7399, 7220, 9022, 6866, 19588, 9077, 18337, 17955, 16913, 13485, 8146, 8196, 9265, 7162, 9018, 7895, 10936, 9034, 8325, 9148, 45887, 9402, 19495, 24117, 8137, 12904, 10590, 11698, 32937, 9351, 7470, 19325, 8702, 11768, 9129, 10542, 19561, 7788, 15709, 9782, 11649, 13023, 9337, 15092, 8092, 9620, 22375, 9076, 9038, 9863, 10578, 15605, 8263, 35453, 11718, 21319, 7532, 15378, 10401, 50997, 9277, 19635, 8075, 11594, 9199, 9929, 6824, 13675, 30903, 11114, 9355, 12517, 43242, 13203, 9134, 18607, 9362, 39376, 43056, 13768, 28569, 24488, 406, 9316, 32010, 23753, 7991, 15525, 37767, 10070, 7235, 10917, 24454, 11387, 35187, 20337, 31994, 9046, 7890, 25226, 9272, 46588, 14485, 9172, 7587, 13486, 9723, 681, 9661, 16691, 8, 12199, 8711, 10033, 13805, 21734, 9563, 19367, 13386]


'예수님께 기도드리고 싶습니다.\n그런데 그게 무슨 소용이 있겠습니까?\n이제 저는 이십 년 전쯤부터 제가 하나님의 말씀을 듣고 있습니다.\n하나님은 우리를 구원하시는 분입니다.\n그래서 우리가 지금 어떻게 해야 할까요?\n우리가 무엇을 원하고 어떤 일을 원하는지 알아야 합니다.\n그러면 우리는 왜 살아야 하는가?\n왜 사는 것이 필요한가요?\n우리는 무엇 때문에 살고 있는가?\n우리에게 주어진 것은 무엇인가?\n그리고 우리의 삶은 무엇인가?라는 질문을 던져야 됩니다.\n저는 오늘도 이렇게 질문합니다.\n예수님이 우리에게 주신 메시지는 무엇일까요?\n바로 ‘믿음의 힘’이었습니다.</d> 지난해 12월 31일 오후 서울 종로구 세종'

In [2]:
# 입력문장의 다음 단어로 가장 확률이 높은 단어 Top 5 예측
output = model(input_ids)
top5 = tf.math.top_k(output.logits[0, -1], k=5)
tokenizer.convert_ids_to_tokens(top5.indices.numpy())

['께', '께서', '과', ',', '께서는']

In [3]:
# Upgrade transformers and datasets to latest versions
'''
pip install tensorflow==2.7.0
pip install transformers==4.21.0
'''

'\npip install tensorflow==2.7.0\npip install transformers==4.21.0\n'

In [7]:
# GPTv2 모델 파인튜닝 & 저장

import tensorflow as tf
from transformers import TFGPT2LMHeadModel, AutoTokenizer
from timeit import default_timer

# Load the text data
with open('bible_john_corpus.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Instantiate the GPT-2 model
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

# Instantiate the tokenizer
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

# Tokenize the text data
tokenized_text = tokenizer(text, return_tensors='tf')
print(tokenized_text)

# Define the training function
@tf.function
def train_step(input_ids):
    # Truncate input sequence
    max_seq_length = 1024 # Usually 1024 for GPT-2
    input_ids = input_ids[:, :max_seq_length]
    with tf.GradientTape() as tape:
        outputs = model(input_ids, training=True)
        logits = outputs.logits[:, :-1, :]
        labels = input_ids[:, 1:]
        loss_value = loss(labels, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return loss_value

# Define the training parameters
batch_size = 16
learning_rate = 3e-5
epochs = 1

# Create a TensorSliceDataset from the tokenized text
dataset = tf.data.Dataset.from_tensor_slices(tokenized_text['input_ids'])
dataset = dataset.batch(batch_size)

# Define the loss function and optimizer
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Fine-tune the model
for epoch in range(epochs):
    start = default_timer()
    print(f'Epoch {epoch+1}/{epochs}')
    for step, batch in enumerate(dataset):
        loss_value = train_step(batch)
        if step % 50 == 0:
            print(f'Step {step} Loss {loss_value}')
    end = default_timer()
    print("Time duration(in seconds):", end - start)
    
    # Save the fine-tuned model
    model.save_pretrained(f'./output/gpt2-finetuned-epoch-{epoch+1}')
    tokenizer.save_pretrained(f'./output/gpt2-finetuned-epoch-{epoch+1}')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.6.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'lm_head.weight', 'transformer.h.11.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.9.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

{'input_ids': <tf.Tensor: shape=(1, 27695), dtype=int32, numpy=array([[ 9724,  7492,  7953, ...,  8705, 32240,  9051]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 27695), dtype=int32, numpy=array([[1, 1, 1, ..., 1, 1, 1]], dtype=int32)>}
Epoch 1/1
Step 0 Loss 3.850346565246582
Time duration(in seconds): 11.203914273006376


In [8]:
# GPT 모델 활용 : 문장 요약
# Note that the model_path variable should be set to the path where the saved model is stored on your machine.

import tensorflow as tf
from transformers import TFGPT2LMHeadModel, AutoTokenizer

# Load the tokenizer and model
model_path = "./output/gpt2-finetuned-epoch-1"
model = TFGPT2LMHeadModel.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Generate summaries
max_length = 40
num_beams = 4
input_text = "예수님"
input_ids = tokenizer.encode(input_text, return_tensors="tf")
output_ids = model.generate(input_ids, max_length=max_length, num_beams=num_beams, no_repeat_ngram_size=2, early_stopping=True)
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Print the generated summary
print("Input Text: ", input_text)
print("Generated Summary: ", output_text)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./output/gpt2-finetuned-epoch-1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Input Text:  예수님
Generated Summary:  예수님께 감사드린다”며 “예수님께서 우리를 사랑하셨기 때문에 우리도 사랑할 수 있었다”고 고백했다.
이어 “우리가 예수님을 사랑하지 않았기 때문에 우리가 우리


In [9]:
# GPT 모델 활용 : 문장 생성
# In this example, we first load the tokenizer and fine-tuned model using their respective paths.
# Then we set the generation parameters: max_length controls the maximum length of the generated text,
# temperature controls the randomness of the text (higher temperature leads to more random and creative output),
# and num_beams controls the number of beams used in beam search for text generation.

import tensorflow as tf
from transformers import TFGPT2LMHeadModel, AutoTokenizer

# Load the tokenizer and model
model_path = "./output/gpt2-finetuned-epoch-1"
model = TFGPT2LMHeadModel.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Set the generation parameters
max_length = 100
temperature = 0.5
num_beams = 4

# GPT가 생성할 문장의 방향성을 알려주기 위한 시작 문자열
sent = '예수님'

# 텍스트 시퀀스를 정수 시퀀스로 변환
input_ids = tokenizer.encode(sent)
input_ids = tf.convert_to_tensor([input_ids])
print(input_ids)

# 정수 시퀀스를 입력받아 GPT가 이어서 문장을 생성 : 약 20초 걸림 (using 1 cpu)
output = model.generate(input_ids, # a tensor containing the input sequence encoded as integer IDs
                        max_length=128, # the maximum length of the generated sequence, in terms of tokens
                        repetition_penalty=2.0, # avoiding repeated tokens (higher value means more diverse output)
                        use_cache=True) # enables or disables the use of the model's internal cache (repetitive output)

output_ids = output.numpy().tolist()[0]
print(output_ids)

# 정수 시퀀스를 텍스트 시퀀스로 변환
tokenizer.decode(output_ids)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./output/gpt2-finetuned-epoch-1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


tf.Tensor([[17835  7177]], shape=(1, 2), dtype=int32)
[17835, 7177, 6995, 16098, 7281, 9885, 13358, 8017, 10010, 6947, 7399, 7220, 9022, 6866, 44224, 14782, 7965, 7172, 7532, 17582, 10401, 14927, 9366, 33866, 9535, 7422, 15433, 42138, 8006, 13675, 22507, 15570, 9564, 9032, 9758, 7916, 12583, 43913, 9394, 9863, 10229, 45023, 19588, 11271, 9651, 19825, 18663, 9176, 12102, 11649, 12341, 15597, 24692, 9207, 9548, 9290, 9054, 38461, 6889, 9108, 11993, 13166, 19520, 13997, 9339, 9348, 9094, 7621, 7415, 7258, 8263, 10171, 16913, 13485, 10972, 10185, 9179, 9661, 34693, 43056, 9063, 12333, 32434, 11106, 18961, 8075, 7080, 9511, 11777, 45937, 14686, 13229, 13953, 24022, 9237, 34487, 19561, 7788, 9258, 10152, 13548, 11382, 7489, 23532, 16522, 8718, 13083, 6969, 10078, 21319, 23775, 9871, 9306, 14208, 7162, 11190, 19730, 7235, 14611, 9801, 15538, 9745, 23236, 38760, 22944, 8704, 9199, 9554]


'예수님께 기도드리고 싶었습니다.\n그런데 그게 아니었어요.\n아니면 내가 왜 그렇게 많은 기도를 드렸는지 모르겠어요?\n내가 얼마나 많이 기도로써 왔는지, 그리고 어떤 사람이 내게 무슨 말을 했는지는 모르지만 나는 어떻게 해서든지 나를 위해 모든 것을 다 바쳤고 또 그것을 통해서 나의 삶을 살도록 내버려두지 않았습니까?\n나는 그런 사람이었죠.\n그리고 나에게는 아무것도 없었지요.\n왜냐하면 그것은 나에게 아무런 도움이 되지 않았기 때문이지요.\n그래서 그는 자기 자신을 위해서만 자기를 희생했으니까요.\n그러니까 그가 다른 사람들에게는 아무 도움도 주지 않고 오직 자신의 삶에 대해서만 헌신한 것이 아니라'